# Variational Autoencoder (VAE) with MNIST

![alt text](http://www.fastforwardlabs.com/blog-images/miriam/imgs_code/vae.4.png "VAE")


## What is VAE?
VAE is a type of generative model that can be used to **a)** take in an image and generate a new image with some desired quality (e.g. of a different style), **b)** generate images after being trained on some input training set, and **c)** understand the "latent" (hidden) factors that cause variations in the images. There are two stages to a VAE, which I will elaborate below.

## Encode Stage
This stage takes a data input vector `x` with dimension `n_in`. For example, this could be a dimension 784 image of a digit. The goal of this stage is to find a lower-dimensional representation for the set of images in general that captures the major variations between images (like the fact that 1 doesn't have a loop but 6 has a loop). This means that the encode phase will try to map an image `x` to a vector `z_mean` in the "latent" space, which contains the important characteristics of `x` in an encoded fashion. As a simple example, if we choose `z_mean` to be a shape (2,) vector, then `z_mean[0]` may quantify how loopy the digit is, and `z_mean[1]` may quantify how slanted the digit is. Obviously `2` characteristics alone will not let us distinguish between all the characters, but if we choose `z_mean`'s dimension (call this `n_latent`) to be around `20` or so, we have no problem encoding the important distinguishing features between the digits.

## Decoding Stage
The decoder takes the "latent" output of the encoder (for now think assume the output is `z_mean`), and attempts output some `y` that is close to a given goal $\hat{y}$. In this particular assignment, $\hat y = x$, meaning that we want the decoder to recover the original input `x` back from the encoding. This entire encoding / decoding process is kinda similar to compression (into the latent space) and trying to recover the original input as well as possible.

## Encoding Stage Revisited
The above is most of the basic idea of a normal (non-variational) autoencoder. One issue with the above is overfitting, i.e. the encoder and decoder will work together to perfectly generate the input that it sees, but fails on some new images from the test set. In that case, the encoder and decoder functions would learn specific mappings from each input $x_i$ in the training set to their respective $z_i$ and back to the original $x_i$, but fail to capture the larger general patterns (e.g. what makes a `0` look like a `0`). To guard against this, the encoding stage is modified as described below.

Instead of modeling the latent space as a single vector `z_mean`, we represent each `z_mean[i]` as the mean of a gaussian distribution, and we also generate a vector `z_std`, so that `z_std[i]` is the standard deviation of the same gaussian distribution. We end up with `n_latent` independent distributions, and the ith distribution is $\mathcal{N}(z_{mean}[i], z_{std}[i])$. The combination of the distribution at each i results in $\mathcal{N}(z_{mean}, z_{std})$, which is called a multivariate gaussian distribution and is a joint distribution of all the $\mathcal{N}(z_{mean}[i], z_{std}[i])$. This means that the original `x` corresponds to a sample of this multivariate distribution. Instead of outputting `z`, the encoder will output $z = z_{mean} + \epsilon * z_{sigma}$, where each element of $\epsilon$ is sampled from $\mathcal{N}(0, 1)$ (the normal distribution with mean 0 and standard deviation 1). This `z` represents another sample from $\mathcal{N}(z_{mean}, z_{std})$, and this `z` rather than `z_mean` is what we feed to the decoder during training, and aim to generate something similar to the original input `x`.

**Don't worry too much about understanding the math**, but hopefully the above didn't confuse you too much. I think the motivation for using a normal distributed output `z` as the output of the encoding stage rather than just using `z_mean` is so that we can group together inputs with slight variations between them. For example, instead of mapping a digit 5 tilted at $15^\circ$ and a digit 5 tilted at $16^\circ$ to two completely different `z_mean`'s, we can now treat them as samples of the distribution $\mathcal{N}(z_{mean}, z_{std})$, which represents the same object but allows for some minor variations.

However, you can check out https://arxiv.org/pdf/1312.6114.pdf or https://arxiv.org/pdf/1606.05908.pdf if you're interested in the math.

## Training the VAE
We will represent the encoder and decoder as neural networks, and apply a loss at the end of the decoder so we can use gradient descent to update the weights to minimize the loss. This loss will be a combination of **a)** a measure of how far the generated output `y` is from the expected output $\hat y = x$ and **b)** a measure of how close the distribution of $z$ is to a standard Gaussian $\mathcal{N}(0, 1)$ (although we could choose a different distribution here). In the picture below, the right side is how our neural network will be structured, and the left side is the what the right side approximates.

![alt text](http://blog.qure.ai/assets/images/vae/Encoder_Decoder_VAE.png "architecture")

## Building the VAE
See inline comments below

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from mnist import MNIST

In [62]:
def load_dataset():
    mndata = MNIST('data/')
    X_train, labels_train = map(np.array, mndata.load_training())
    X_test, labels_test = map(np.array, mndata.load_testing())
    X_train, X_test = X_train / 255.0, X_test / 255.0
    return X_train, labels_train, X_test, labels_test

X_train, labels_train, X_test, labels_test = load_dataset()

In [70]:
class VAE:
    def __init__(self, n_in, n_latent, n_out, learning_rate=1e-3):
        '''
        Essentially we hook up one net for the encoder and one for the decoder.
        
        Encoder:
        x is an input image. z_mean, z_log_std are predicted from the encoder 
        and is used to create z, along with epsilon (which is sampled from a 
        random normal distribution)
            x -- (encoder neural network) -- z_mean ---- z = z_mean + epsilon * z_std
                                          \- z_log_std --/ /
             sample random normal N(0, 1) -- epsilon -----/
        
        Your choices of variable will be:
            size of latent space (dimension of z_mean and z_log_std) - just choose n_latent = 20 for now
            structure of encoder: You can try 
                x (n_in) -> FC (n_in to 500) -> relu -> FC (500 to 300) -> relu -> FC (300 to n_latent) -> z_mean (n_latent)
                                                                               \-> FC (300 to n_latent) -> z_log_std (n_latent)
            where FC = fully connected. You can replace ReLU with some other activation function too if you want.
            For the FC layers, initialize the weights with Xavier initialization and the biases to 0.
            Notice that we are predicting the log of the std with this network, so to get std, you need to use 
            tf.exp(). Use tf.random_normal to generate epsilon.
            
            TRY NOT TO HARD CODE THE LAYER SIZES (it's useful to make these parameters of the __init__ so you can
            specify them when you initialize the network)
        '''
        self.n_z = n_z
        self.x = tf.placeholder(tf.float32, shape=(None, n_in))
        
        self.z_mean = # tensorflow expression
        z_log_sq_sigma = # tensorflow expression

        eps = # use tf.random_normal()
        self.z = 
        
        
        '''
        Decoder:
            here n_out = n_in since we're trying to recreate the input
            z (n_latent) -> FC (n_z to 300) -> relu -> FC (300 to 500) -> relu -> FC (500 to n_out) -> tf.nn.sigmoid -> y
        '''
        self.y = # output of decoder
        
        '''
        Loss:
        decode loss is the sum of how much output pixels deviates from the input pixels, the expression is:
            -sum_i (x[i] * log(1e-10 + y[i]) + (1 - x[i]) log(1e-10 + 1 - y[i])),   0 <= i < n_in
        You should do this in a VECTORIZED fashion. i.e. calculate a vector v then use tf.reduce_sum(v, axis=1)
        
        latent loss is how far the z distribution deviates from a standard normal, the expression is:
            -sum_j (1 + log(z_sigma^2) - z_mean^2 - z_sigma^2),   0 <= j < n_latent
        again, do this with vectorization
        '''
        decode_loss = 
        latent_loss = 
        
        # the total loss is the average of (decode_loss + latent_loss) over the entrie batch, so use tf.reduce_mean()
        self.loss = #
        
        # Doesn't really matter which optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)
        
        # create tensorflow session here
        self.sess =
        
    def fit_batch(self, x):
        '''
        feeds in a shape (batch, n_in) x into the network and do backprop
        ''' 
        _, loss = self.sess.run((self.optimizer, self.loss), feed_dict={self.x: x})
        return loss
    
    def encode(self, x):
        '''
        feeds in a shape (batch, n_in) x into the network and ONLY calculate self.z_mean (encoder phase only)
        DO NOT DO backprop (aka don't pass in self.optimizer into self.sess.run)
        if you coded your network flexibly, you don't need batch to equal n_batch (the batch size during training)
        Hint: use self.sess.run again like in fit_batch, but modify a little
        '''
        return # fill me in
    
    def decode(self):
        '''
        feeds in a shape (batch, n_latent) z into the network and ONLY calculate the output (decoder phase only)
        DO NOT DO backprop
        
        Essentially this allows you to generate a digit by feeding in a normal distribution z with unit standard variation!
        '''
        return 
    
    def reconstruct(self, x):
        '''
        feeds in a shape (batch, n_in) x into the network and calculate self.y
        DO NOT DO backprop
        '''
        return 
    
    def train(self, X, batch_size, epochs, display_step=5):
        '''
        You should understand this code and be able to write this if you need to
        '''
        X = np.random.permutation(X)
        n_samples = len(X)
        mean_losses = []
        for epoch in range(epochs):
            total_iter = n_samples // batch_size
            total_loss = 0
            for i in range(total_iter):
                x = X[i * batch_size : (i + 1) * batch_size]
                loss = self.fit_batch(x)
                total_loss += loss
            mean_loss = total_loss / total_iter
            mean_losses.append(mean_loss)
            if (epoch + 1) % display_step == 0:
                print('epoch %s: loss=%.4f' % (epoch + 1, mean_loss))

In [ ]:
vae = # initialize VAE
vae.train(X_train, 100, 25) # train with batch_size of 100 and 25 epochs

In [26]:
def graph(x_pred, x_true):
    '''
    graphs one predicted and one original x side by side
    '''
    for i, x in enumerate([x_pred, x_true]):
        # use plt.subplot to put two plots side by side. Look this up!
        plt.subplot
        # use plt.imshow on a reshaped x, try the arguments cmap='gray' to use grayscale and interpolation='nearest'
        plt.imshow
        plt.axis('off')
    plt.show()

In [ ]:
# predict a few images from the test and training set, do the output look realistic?
vae.reconstruct()

## Latent Space Visualization
We will look at the latent space to understand it a little better

In [ ]:
# retrain a vae with n_latent = 2
vae_2d =
vae_2d.train(X_train, 100, 15)

# get 1000 samples from X_train, use vae_2d.encode to encode them into 2D latent space
# plot the encoded points with color equal to their label (digit value)
plt.scatter(x, y, c=labels)
plt.colorbar() # show the color bar
plt.show()
# you should be able to see clusters of points corresonding to each label in the latent space

In [ ]:
# now we will try feed in z_mean in a systematic way and use vae.decode to visualize what each z_mean generates
nx = ny = 20
# generates 20 evenly spaced x_values and y_values for the z_mean
x_values = np.linspace(-3, 3, nx)
y_values = np.linspace(-3, 3, ny)

canvas = np.zeros((28 * ny, 28 * nx)) # a grid that will contain 20 x 20 digit images
for i, yi in enumerate(x_values):
    for j, xi in enumerate(y_values):
        z_mean = np.array([[xi, yi]])
        # decode z_mean and add the result to the canvas
plt.imshow(canvas, cmap='gray')